# StateLog × TradeLog 解析テンプレート
- `analysis/env_data` と `analysis/bt_results` に配置した CSV を読み込み、環境指標とBT結果を突き合わせるためのノートです。
- `merged_trades` DataFrame を作成しておけば、帯域別・時間別の集計を容易に反復できます。

In [2]:
import pandas as pd
from pathlib import Path
DATA_ROOT = Path('analysis')
ENV_DIR = DATA_ROOT / 'env_data'
BT_DIR = DATA_ROOT / 'bt_results'
print('Env files:', list(ENV_DIR.glob('*.csv')))
print('BT files:', list(BT_DIR.glob('*.csv')))

Env files: [PosixPath('analysis/env_data/StateLog_USDJPY_M5_20250509.csv'), PosixPath('analysis/env_data/StateLog_USDJPY_M5_20250929.csv'), PosixPath('analysis/env_data/StateLog_USDJPY_M5_20251028.csv'), PosixPath('analysis/env_data/StateLog_USDJPY_M5_20250125.csv'), PosixPath('analysis/env_data/StateLog_USDJPY_M5_20250505.csv'), PosixPath('analysis/env_data/StateLog_USDJPY_M5_20250916.csv'), PosixPath('analysis/env_data/StateLog_USDJPY_M5_20250513.csv'), PosixPath('analysis/env_data/StateLog_USDJPY_M5_20251111.csv'), PosixPath('analysis/env_data/StateLog_USDJPY_M5_20250623.csv'), PosixPath('analysis/env_data/StateLog_USDJPY_M5_20250131.csv'), PosixPath('analysis/env_data/StateLog_USDJPY_M5_20251008.csv'), PosixPath('analysis/env_data/StateLog_USDJPY_M5_20250703.csv'), PosixPath('analysis/env_data/StateLog_USDJPY_M5_20250210.csv'), PosixPath('analysis/env_data/StateLog_USDJPY_M5_20251023.csv'), PosixPath('analysis/env_data/StateLog_USDJPY_M5_20250614.csv'), PosixPath('analysis/env_data

In [3]:
# ==== 自動で環境データを取得して突き合わせる ====
bt_file = BT_DIR / 'AtrBandConfig_YoYoEA_Multi_Entry_Test_Guard_20250901_20251115.csv'
bt_df = pd.read_csv(bt_file)
bt_df['timestamp'] = pd.to_datetime(bt_df['timestamp'].str.replace('.', '-', regex=False))

# BTに含まれる取引日の環境ファイルを自動収集
from collections import defaultdict
date_keys = sorted(bt_df['timestamp'].dt.strftime('%Y%m%d').unique())
by_date = defaultdict(list)
# env_data配下を一度だけ走査
for csv_path in ENV_DIR.glob('*.csv'):
    key = ''.join(filter(str.isdigit, csv_path.stem))[-8:]
    by_date[key].append(csv_path)

env_frames = []
for key in date_keys:
    candidates = by_date.get(key)
    if not candidates:
        print(f'[WARN] env file not found for {key}')
        continue
    env_part = pd.concat([pd.read_csv(p, sep=';') for p in candidates], ignore_index=True)
    env_part['bar_time'] = pd.to_datetime(env_part['bar_time'].str.replace('.', '-', regex=False))
    env_frames.append(env_part)

if not env_frames:
    raise FileNotFoundError('No environment files matched BT dates.')

env_df = pd.concat(env_frames, ignore_index=True).sort_values('bar_time')
print('Loaded env rows:', len(env_df), 'from', len(env_frames), 'filesets')

entries = bt_df[bt_df['event'] == 'ENTRY'].copy().sort_values('timestamp')
merged = pd.merge_asof(entries, env_df, left_on='timestamp', right_on='bar_time',
                      direction='nearest', tolerance=pd.Timedelta('5min'))
print('merged rows', len(merged))
merged.head()


Loaded env rows: 15614 from 55 filesets
merged rows 766


,timestamp_x,run_id_x,event,symbol_x,profile_x,strategy,direction,ticket,volume,price,...,ma_long,ma_slope,bb_width,donchian_width,fibo_ratio,spread,session,weekday,volatility,notes
0,2025-09-01 02:55:00,20250901_010100,ENTRY,USDJPY,test\AtrBandConfig_YoYoEA_Multi_Entry_Test_Gua...,RSI,SELL,1,0.1,147.129,...,147.040405,0.587,0.0,0.167,0.844311,0.5,ASIA,1,LOW,NaN
1,2025-09-01 05:05:00,20250901_010100,ENTRY,USDJPY,test\AtrBandConfig_YoYoEA_Multi_Entry_Test_Gua...,STOCH,SELL,2,0.1,147.198,...,147.055420,0.375,0.0,0.205,0.897561,0.5,ASIA,1,LOW,NaN
2,2025-09-01 05:25:00,20250901_010100,ENTRY,USDJPY,test\AtrBandConfig_YoYoEA_Multi_Entry_Test_Gua...,RSI,SELL,3,0.1,147.330,...,147.063825,0.722,0.0,0.280,0.964286,0.5,ASIA,1,LOW,NaN
3,2025-09-01 06:35:00,20250901_010100,ENTRY,USDJPY,test\AtrBandConfig_YoYoEA_Multi_Entry_Test_Gua...,STOCH,BUY,4,0.1,147.047,...,147.076545,-0.772,0.0,0.354,0.067797,0.5,EUROPE,1,LOW,NaN
4,2025-09-01 07:00:00,20250901_010100,ENTRY,USDJPY,test\AtrBandConfig_YoYoEA_Multi_Entry_Test_Gua...,STOCH,BUY,5,0.1,146.891,...,147.072545,-1.489,0.0,0.541,0.127542,0.5,EUROPE,1,LOW,NaN


In [4]:
# ==== 例: ATR帯域×戦略の損益集計 ====
exits = bt_df[bt_df['event'] == 'EXIT'].copy()
exits['atr_entry'] = exits['atr_entry'].astype(float)
bins = [0, 0.05, 0.08, 0.10, 0.12, 0.20, 1]
labels = ['0.00-0.05','0.05-0.08','0.08-0.10','0.10-0.12','0.12-0.20','0.20+']
exits['atr_band'] = pd.cut(exits['atr_entry'], bins=bins, labels=labels, right=False)
pivot = exits.pivot_table(index='atr_band', columns='strategy', values='net', aggfunc='sum', fill_value=0)
pivot

/tmp/ipykernel_57436/318773033.py:7: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot = exits.pivot_table(index='atr_band', columns='strategy', values='net', aggfunc='sum', fill_value=0)


strategy,CCI,MACD,MA_CROSS,RSI,STOCH
atr_band,,,,,
0.00-0.05,0.00,0.0,0.00,-5.01,0.00
0.05-0.08,0.00,0.0,-39.61,0.00,-167.52
0.08-0.10,0.00,0.0,-13.06,-22.76,0.00
0.10-0.12,0.00,0.0,26.47,0.00,0.00
0.12-0.20,-97.01,0.0,-3.42,0.00,0.00
0.20+,40.77,7.4,41.18,-43.71,0.00


## TODO
- 週次/日次での State×BT 結合関数を関数化する
- 追加の特徴量（ATR増減、セッションフラグなど）を派生列として定義する
- ML モデル用に `merged` から特徴量テーブルを作成する